In [2]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
parentdir = os.path.dirname(parentdir)
sys.path.insert(0,parentdir)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import sys
sys.path.append('/content/drive/My Drive/D2FM')
module_path = '/content/drive/My Drive/D2FM'
if module_path not in sys.path:
    sys.path.append(module_path)

In [5]:
!pip install pykalman

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 5.4 MB/s eta 0:00:00


In [6]:
from models.ddfm import *

In [7]:
import numpy as np
import pandas as pd
import tensorflow as tf
from typing import Tuple
from tensorflow import keras
from tensorflow.keras import layers
from statsmodels.tsa.statespace.dynamic_factor_mq import DynamicFactorMQ as DFM
from sklearn.model_selection import ParameterGrid
from typing import Tuple
import numpy as np
from pykalman import KalmanFilter

In [8]:
#from google.colab import files
#uploaded = files.upload()

In [9]:
import warnings
warnings.filterwarnings("ignore")
import random
from sklearn.metrics import mean_squared_error as mse
from typing import Tuple
from sklearn.preprocessing import StandardScaler

In [10]:
df_train = pd.read_csv('/content/drive/MyDrive/D2FM/scaled_train.csv')
df_test=pd.read_csv('/content/drive/MyDrive/D2FM/scaled_test.csv')


In [11]:
df_train.columns

Index(['Index', 'RPI', 'W875RX1', 'DPCERA3M086SBEA', 'INDPRO', 'IPFPNSS',
       'IPFINAL', 'IPCONGD', 'IPDCONGD', 'IPNCONGD',
       ...
       'AAAFFM', 'BAAFFM', 'EXSZUSx', 'EXJPUSx', 'EXUSUKx', 'EXCAUSx',
       'DTCOLNVHFNM', 'DTCTHFNM', 'INVEST', 'FEDFUNDS'],
      dtype='object', length=119)

In [12]:
df_train_idx=df_train.set_index("Index")
df_test_idx=df_test.set_index("Index")

Minimum Loss: 0.6215284234293389 with encoder (120, 60, 40, 5), lag 1, use_bias False, link tanh, batch_size 72, r 5

Minimum Loss: 0.589956869321811 with encoder (144, 72, 48, 6), lag 1, use_bias True, link tanh, batch_size 100, r 6

Minimum Loss: 0.560155765006405 with encoder (168, 84, 56, 7), lag 1, use_bias True, link tanh, batch_size 72, r 7

## Para 5 factores:

In [13]:
ddfm5 = DDFM(df_train_idx, structure_encoder=(120, 60, 40, 5), lags_input=1, factor_oder=2,
                             use_bias=False, link='tanh', batch_size=72, max_iter=1000)
ddfm5.fit()

@Info - Note: Sorting data.
18/18 [==============================] - 0s 2ms/step
@Info: Convergence achieved in 5 iterations - new loss: 0.6220574668162808 - delta: 0.0004976296843081726 < 0.0005


In [14]:
last_state, last_state_var_cov_matrix = ddfm5.get_last_state()

In [15]:
steps_ahead = df_test_idx.shape[0]  # for example, predict 5 steps ahead
predictions5 = ddfm5.predict(last_state, last_state_var_cov_matrix, steps_ahead)
#predictions7['states'].shape
y_t_k_5=ddfm5.predict_observations(last_state, last_state_var_cov_matrix , steps_ahead)
#df_test_first_12_rows = df_test_idx.iloc[:12, :]
test_array = df_test_idx.to_numpy()
mse_row=np.mean((test_array - y_t_k_5)**2, axis=1)
mse_145=np.mean(mse_row)
mae_145 = np.mean(np.abs(test_array - y_t_k_5))
rmsfe_145 = np.sqrt(np.mean((test_array - y_t_k_5) ** 2))
print("Mean Squared Error:", mse_145)
print("Mean Absolute Error:", mae_145)
print("Root Mean Squared Forecast Error:", rmsfe_145)

Mean Squared Error: 0.9639020017473933
Mean Absolute Error: 0.6861313875072609
Root Mean Squared Forecast Error: 0.9817851097604778


## Para 6 factores:

In [16]:
ddfm6 = DDFM(df_train_idx, structure_encoder=(144, 72, 48, 6), lags_input=1, factor_oder=2,
                             use_bias=True, link='tanh', batch_size=100, max_iter=1000)
ddfm6.fit()

@Info - Note: Sorting data.
18/18 [==============================] - 0s 2ms/step
@Info: Convergence achieved in 5 iterations - new loss: 0.5897969142813294 - delta: 0.0004564516147610657 < 0.0005


In [17]:
last_state, last_state_var_cov_matrix = ddfm6.get_last_state()

In [18]:
last_state.shape

(130,)

In [19]:
steps_ahead = df_test_idx.shape[0]  # for example, predict 5 steps ahead
predictions6 = ddfm6.predict(last_state, last_state_var_cov_matrix, steps_ahead)
#predictions7['states'].shape
y_t_k_6=ddfm6.predict_observations(last_state, last_state_var_cov_matrix , steps_ahead)
#df_test_first_12_rows = df_test_idx.iloc[:12, :]
test_array = df_test_idx.to_numpy()
mse_row=np.mean((test_array - y_t_k_6)**2, axis=1)
mse_145=np.mean(mse_row)
mae_145 = np.mean(np.abs(test_array - y_t_k_6))
rmsfe_145 = np.sqrt(np.mean((test_array - y_t_k_6) ** 2))
print("Mean Squared Error:", mse_145)
print("Mean Absolute Error:", mae_145)
print("Root Mean Squared Forecast Error:", rmsfe_145)

Mean Squared Error: 0.9575909769901686
Mean Absolute Error: 0.6841686213413235
Root Mean Squared Forecast Error: 0.9785657755052383


## Para 7 factores

In [20]:
ddfm7 = DDFM(df_train_idx, structure_encoder=(168, 84, 56, 7), lags_input=1, factor_oder=2,
                             use_bias=True, link='tanh', batch_size=72, max_iter=1000)
ddfm7.fit()
last_state, last_state_var_cov_matrix = ddfm7.get_last_state()
steps_ahead = df_test_idx.shape[0]  # for example, predict 5 steps ahead
predictions7 = ddfm7.predict(last_state, last_state_var_cov_matrix, steps_ahead)
#predictions7['states'].shape
y_t_k_7=ddfm7.predict_observations(last_state, last_state_var_cov_matrix , steps_ahead)
#df_test_first_12_rows = df_test_idx.iloc[:12, :]
test_array = df_test_idx.to_numpy()
mse_row=np.mean((test_array - y_t_k_7)**2, axis=1)
mse_145=np.mean(mse_row)
mae_145 = np.mean(np.abs(test_array - y_t_k_7))
rmsfe_145 = np.sqrt(np.mean((test_array - y_t_k_7) ** 2))
print("Mean Squared Error:", mse_145)
print("Mean Absolute Error:", mae_145)
print("Root Mean Squared Forecast Error:", rmsfe_145)

@Info - Note: Sorting data.
18/18 [==============================] - 0s 2ms/step
@Info: Convergence achieved in 6 iterations - new loss: 0.5602274381179211 - delta: 0.0003571184439098407 < 0.0005
Mean Squared Error: 0.9564052896107641
Mean Absolute Error: 0.6838071369116704
Root Mean Squared Forecast Error: 0.9779597586868102


In [ ]:
#n = ddfm.state_space.F.shape[0]
#x_hat_start = np.zeros((1, n))
#sigma_x_hat_start = np.eye(ddfm.state_space.F.shape[0])
#x_hat_start.shape

In [ ]:
#dir(ddfm)

### factores filtrados

In [21]:
ff5=ddfm5.factors_filtered
f_hat_ddfm_idx5 = pd.DataFrame(ff5, columns=["f1", "f2", "f3", "f4", "f5"])
f_hat_ddfm_idx5.index = df_train_idx.index
f_hat_ddfm5=f_hat_ddfm_idx5.reset_index()
f_hat_ddfm5.to_csv('/content/drive/MyDrive/D2FM/f_hat_DDFM_fact5.csv', index=False)

In [22]:
ff6=ddfm6.factors_filtered
f_hat_ddfm_idx6 = pd.DataFrame(ff6, columns=["f1", "f2", "f3", "f4", "f5", "f6"])
f_hat_ddfm_idx6.index = df_train_idx.index
f_hat_ddfm6=f_hat_ddfm_idx6.reset_index()
f_hat_ddfm6.to_csv('/content/drive/MyDrive/D2FM/f_hat_DDFM_fact6.csv', index=False)

In [23]:
ff7=ddfm7.factors_filtered
ff7.shape

(577, 7)

In [24]:
f_hat_ddfm_idx7 = pd.DataFrame(ff7, columns=["f1", "f2", "f3", "f4", "f5", "f6", "f7"])

In [25]:
f_hat_ddfm_idx7.head()

,f1,f2,f3,f4,f5,f6,f7
0,-0.436209,-0.313294,0.678699,0.477978,0.371764,0.213296,0.000000
1,0.152070,0.422210,-0.203465,0.262834,-0.634552,-0.134829,-0.554202
2,-0.445482,0.138910,-0.110341,0.070396,-0.577147,-0.443397,-0.489403
3,0.511943,0.355700,-0.360430,0.336868,-0.148251,-0.704480,-0.244418
4,-0.678348,0.172320,0.164158,-0.049188,0.006871,-0.061646,-0.289158


In [26]:
f_hat_ddfm_idx7.index = df_train_idx.index
f_hat_ddfm7=f_hat_ddfm_idx7.reset_index()

In [27]:
f_hat_ddfm7.head()

,Index,f1,f2,f3,f4,f5,f6,f7
0,1960-01-01,-0.436209,-0.313294,0.678699,0.477978,0.371764,0.213296,0.000000
1,1960-02-01,0.152070,0.422210,-0.203465,0.262834,-0.634552,-0.134829,-0.554202
2,1960-03-01,-0.445482,0.138910,-0.110341,0.070396,-0.577147,-0.443397,-0.489403
3,1960-04-01,0.511943,0.355700,-0.360430,0.336868,-0.148251,-0.704480,-0.244418
4,1960-05-01,-0.678348,0.172320,0.164158,-0.049188,0.006871,-0.061646,-0.289158


In [28]:
f_hat_ddfm7.to_csv('/content/drive/MyDrive/D2FM/f_hat_DDFM_fact7.csv', index=False)